# SpaceX API Data Ingestion and SQL Analysis

In this exercise, you will fetch data from the SpaceX API, store it in an SQLite database, and analyze it using SQL.

## Objectives

1. Understand the relevant SpaceX API endpoints.
2. Fetch data from the selected endpoints.
3. Store the data in an SQLite database.
4. Write SQL queries to analyze the stored data.

## Step 1: Understand the SpaceX API Endpoints

You are tasked with answering the following questions:

1. **Most Commonly Used Rocket:** Which rocket has been used the most in SpaceX launches?
2. **Most Active Month for Launches:** Which month has the highest number of SpaceX launches?
3. **Breakdown of Astronauts by Agency:** How many astronauts, grouped by agency, have been involved in SpaceX crewed missions?

To answer these questions, you will need to use the following SpaceX API endpoints:

- **Launches:** `/v4/launches`
- **Rockets:** `/v4/rockets`
- **Crew:** `/v4/crew`

You will fetch data from these endpoints, store it in an SQLite database, and then write SQL queries to extract the necessary information.

## Step 2: Fetch and Store Launch Data

First, you need to fetch data about SpaceX launches. This data will help you answer the questions about the most commonly used rocket and the most active month for launches.

You will store this data in a table called `launches` with the following schema:

- `launch_id` (TEXT PRIMARY KEY)
- `rocket_id` (TEXT)
- `launch_date` (TEXT)

Let's start by fetching the launch data and storing it.

In [ ]:
import requests
import sqlite3
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

# Fetch launch data
launches_url = "https://api.spacexdata.com/v4/launches"
launches_response = requests.get(launches_url)
launches_data = launches_response.json()

# Connect to SQLite database
conn = sqlite3.connect('spacex_data.db')
c = conn.cursor()

# Create launches table
c.execute('''
    CREATE TABLE IF NOT EXISTS launches (
        launch_id TEXT PRIMARY KEY,
        rocket_id TEXT,
        launch_date TEXT
    )
''')

# Insert launch data into the database
for launch in launches_data:
    c.execute('''
        INSERT OR IGNORE INTO launches (launch_id, rocket_id, launch_date)
        VALUES (?, ?, ?)
    ''', (launch['id'], launch['rocket'], launch['date_utc']))

conn.commit()
logging.info("Launch data stored successfully.")


## Step 3: Fetch and Store Rocket Data

Next, you need to fetch data about SpaceX rockets. This data will help you identify which rocket has been used the most in SpaceX launches.

You will store this data in a table called `rockets` with the following schema:

- `rocket_id` (TEXT PRIMARY KEY)
- `rocket_name` (TEXT)

Let's fetch the rocket data and store it.

In [ ]:
# Fetch rocket data
rockets_url = "https://api.spacexdata.com/v4/rockets"
rockets_response = requests.get(rockets_url)
rockets_data = rockets_response.json()

# Create rockets table
c.execute('''
    CREATE TABLE IF NOT EXISTS rockets (
        rocket_id TEXT PRIMARY KEY,
        rocket_name TEXT
    )
''')

# Insert rocket data into the database
for rocket in rockets_data:
    c.execute('''
        INSERT OR IGNORE INTO rockets (rocket_id, rocket_name)
        VALUES (?, ?)
    ''', (rocket['id'], rocket['name']))

conn.commit()
logging.info("Rocket data stored successfully.")


## Step 4: Fetch and Store Crew Data

Finally, you need to fetch data about SpaceX crew members. This data will help you break down the number of astronauts by their agency.

You will store this data in a table called `crew` with the following schema:

- `crew_id` (TEXT PRIMARY KEY)
- `name` (TEXT)
- `agency` (TEXT)

Let's fetch the crew data and store it.

In [ ]:
# Fetch crew data
crew_url = "https://api.spacexdata.com/v4/crew"
crew_response = requests.get(crew_url)
crew_data = crew_response.json()

# Create crew table
c.execute('''
    CREATE TABLE IF NOT EXISTS crew (
        crew_id TEXT PRIMARY KEY,
        name TEXT,
        agency TEXT
    )
''')

# Insert crew data into the database
for member in crew_data:
    c.execute('''
        INSERT OR IGNORE INTO crew (crew_id, name, agency)
        VALUES (?, ?, ?)
    ''', (member['id'], member['name'], member['agency']))

conn.commit()
logging.info("Crew data stored successfully.")


## Step 5: Analyze the Data

Now that you have stored all the necessary data, it's time to analyze it and answer the following questions:

1. **Most Commonly Used Rocket:** Which rocket has been used the most in SpaceX launches?
2. **Most Active Month for Launches:** Which month has the highest number of SpaceX launches?
3. **Breakdown of Astronauts by Agency:** How many astronauts, grouped by agency, have been involved in SpaceX crewed missions?

Let's start by writing SQL queries to answer these questions.

In [ ]:
# Query 1: Most Commonly Used Rocket
def query_most_common_rocket():
    c.execute('''
        SELECT rockets.rocket_name, COUNT(launches.rocket_id) as launch_count
        FROM launches
        JOIN rockets ON launches.rocket_id = rockets.rocket_id
        GROUP BY rockets.rocket_name
        ORDER BY launch_count DESC
        LIMIT 1;
    ''')
    result = c.fetchone()
    print(f'Most Commonly Used Rocket: {result[0]} with {result[1]} launches')

query_most_common_rocket()


In [ ]:
# Query 2: Most Active Month for Launches
def query_most_active_month():
    c.execute('''
        SELECT strftime('%Y-%m', launch_date) as month, COUNT(*) as launch_count
        FROM launches
        GROUP BY month
        ORDER BY launch_count DESC
        LIMIT 1;
    ''')
    result = c.fetchone()
    print(f'Most Active Month: {result[0]} with {result[1]} launches')

query_most_active_month()


In [ ]:
# Query 3: Breakdown of Astronauts by Agency
def query_astronauts_by_agency():
    c.execute('''
        SELECT agency, COUNT(*) as astronaut_count
        FROM crew
        GROUP BY agency
        ORDER BY astronaut_count DESC;
    ''')
    results = c.fetchall()
    print('Breakdown of Astronauts by Agency:')
    for result in results:
        print(f'{result[0]}: {result[1]} astronauts')

query_astronauts_by_agency()


## Conclusion

In this notebook, you successfully:

1. Fetched data from the SpaceX API using multiple endpoints.
2. Stored the fetched data in an SQLite database.
3. Analyzed the data to answer key questions about SpaceX's rockets, launches, and crew members.

This exercise demonstrates how to work with real-world APIs, manage data storage, and perform data analysis using SQL.

In [ ]:
# Close the database connection
conn.close()
logging.info('Database connection closed.')
